In [1]:
from HEEM_VQE import *
from qiskit.providers.ibmq.runtime import UserMessenger
from qiskit import Aer
import matplotlib.pyplot as plt

In [2]:
meta = {
  "name": "HEEM_VQE",
  "description": "Variationa Quantum Eigensolver with hardware efficient entangled measurements",
  "max_execution_time": 50000
}

In [3]:
meta["parameters"] = [
    {"name": "iters", "description": "number of iterations, default=10", "type": "int", "required": False },
    {"name": "pars", "description": "Initial vector of parameters.", "type": "ndarray", "required": False}, 
    {"name": "shots", "description": "Number of shots to take per circuit, default=8192", "type": "int", "required": False},
    {"name": "grouping", "description": "Method for grouping the local observables of the Hamiltonian into compatible measurements: tensor product basis ('TPB') or entangled measurements ('Entangled') , default='TPB'.", "type": "str", "required": False},
    {"name": "conectivity", "description": "Device conectivity.", "type": "list", "required": False}
  ]


meta['return_values'] = [
    {"name": "result", "description": "Final result in a dictionary.", "type": "dict"}
  ]

meta["interim_results"] = [
    {"name": "params", "description": "Energy evaluations", "type": "ndarray"},
  ]


In [4]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 

In [5]:
program_id = 'heem-vqe'
provider.runtime.update_program(data='HEEM_VQE.py', metadata=meta, program_id=program_id)

In [6]:
prog = provider.runtime.program(program_id)
print(prog)

heem-vqe:
  Name: HEEM_VQE
  Description: Variationa Quantum Eigensolver with hardware efficient entangled measurements
  Creation date: 2021-10-16T20:48:20Z
  Update date: 2021-11-12T14:15:16.169201Z
  Max execution time: 50000
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [7]:
backend = provider.backend.ibmq_qasm_simulator
# backend = provider.get_backend('ibm_lagos')
options = {'backend_name': backend.name()}

In [8]:
#conectivity = get_backend_conectivity(backend)

In [9]:
inputs = { "iters": 100 ,
           "grouping": "Entangled",
           "shots" : 2**13}

In [10]:
interm_results = []
def vqe_callback(job_id, data):
    interm_results.append(data)

In [11]:
job = provider.runtime.run(program_id, options=options, inputs=inputs, callback=vqe_callback)

In [12]:
job.result()

RuntimeJobFailureError: 'Unable to retrieve job result. Job c677dt737kiphhdbflm0 has failed:\n2021-11-12T14:15:29.117373210Z /usr/local/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than \'r\' is deprecated. Pass the mode to h5py.File() instead.\n2021-11-12T14:15:29.117409592Z   h5py.get_config().default_file_mode = \'a\'\n2021-11-12T14:15:29.117418421Z /usr/local/lib/python3.8/site-packages/sympy/core/expr.py:3949: SymPyDeprecationWarning: \n2021-11-12T14:15:29.117424979Z \n2021-11-12T14:15:29.117421944Z Couldn\'t find cython becke routine\n2021-11-12T14:15:29.117431749Z expr_free_symbols method has been deprecated since SymPy 1.9. See\n2021-11-12T14:15:29.117453861Z https://github.com/sympy/sympy/issues/21494 for more info.\n2021-11-12T14:15:29.117474711Z \n2021-11-12T14:15:29.117482849Z   SymPyDeprecationWarning(feature="expr_free_symbols method",\n2021-11-12T14:15:29.547440366Z /usr/local/lib/python3.8/site-packages/nats/aio

In [ ]:
energies= [ idx['energy'] for idx in interm_results ]

In [ ]:
energies

In [ ]:
plt.plot(energies)
plt.grid()

In [ ]:
plt.plot(energies)
plt.grid()